# Modern Robotics — Rotation representations demo  
## Rotation matrix $R$ ↔ Axis–angle $(\hat{\omega},\theta)$ ↔ Quaternion $q$

Standalone notebook (no project imports). It shows:

1. Build a rotation matrix $R$ from yaw–pitch–roll (composition).
2. Convert $R \rightarrow (\hat{\omega},\theta)$ and reconstruct $R$ via Rodrigues.
3. Convert $R \rightarrow q$ and reconstruct $R$ from quaternion.
4. Visualize axis $\hat{\omega}$, a vector $v$, and the rotated vector $Rv$.
5. (Optional) Save a GIF animation rotating about the recovered axis.

> Tip: if LaTeX doesn’t render in VS Code, trust the notebook (Command Palette → “Trust Notebook”).


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from pathlib import Path

def Rx(a):
    c, s = np.cos(a), np.sin(a)
    return np.array([[1,0,0],[0,c,-s],[0,s,c]])

def Ry(a):
    c, s = np.cos(a), np.sin(a)
    return np.array([[c,0,s],[0,1,0],[-s,0,c]])

def Rz(a):
    c, s = np.cos(a), np.sin(a)
    return np.array([[c,-s,0],[s,c,0],[0,0,1]])

def set_axes_equal_3d(ax, lim=1.2):
    ax.set_xlim(-lim, lim); ax.set_ylim(-lim, lim); ax.set_zlim(-lim, lim)
    try: ax.set_box_aspect([1,1,1])
    except Exception: pass

def draw_arrow(ax, v, color="k", label=None, lw=2, alpha=1.0):
    v = np.asarray(v, dtype=float)
    ax.quiver(0,0,0, v[0],v[1],v[2], color=color, linewidth=lw, alpha=alpha, arrow_length_ratio=0.12)
    if label is not None:
        ax.text(v[0], v[1], v[2], label, color=color)

def draw_frame(ax, R=np.eye(3), name="", axis_len=1.0, colors=("tab:red","tab:green","tab:blue"), lw=2, alpha=1.0):
    R = np.asarray(R, dtype=float)
    ex, ey, ez = R[:,0], R[:,1], R[:,2]
    draw_arrow(ax, axis_len*ex, color=colors[0], label=(f"{name}x" if name else "x"), lw=lw, alpha=alpha)
    draw_arrow(ax, axis_len*ey, color=colors[1], label=(f"{name}y" if name else "y"), lw=lw, alpha=alpha)
    draw_arrow(ax, axis_len*ez, color=colors[2], label=(f"{name}z" if name else "z"), lw=lw, alpha=alpha)

def is_so3(R, atol=1e-6):
    R = np.asarray(R, dtype=float)
    return np.allclose(R.T @ R, np.eye(3), atol=atol) and np.isclose(np.linalg.det(R), 1.0, atol=atol)

print("Ready.")

## 1) Build a rotation matrix $R$ from yaw–pitch–roll
We use the common (extrinsic) composition:
$$R = R_z(\text{yaw})\,R_y(\text{pitch})\,R_x(\text{roll})$$
Order matters.


In [ ]:
yaw_deg, pitch_deg, roll_deg = 40, 20, -10
yaw, pitch, roll = np.deg2rad([yaw_deg, pitch_deg, roll_deg])

R = Rz(yaw) @ Ry(pitch) @ Rx(roll)
print("R =\n", np.round(R, 6))
print("is_so3(R)?", is_so3(R))

## 2) Axis–angle / exponential coordinates (Rodrigues)
A rotation can be represented by a unit axis $\hat{\omega}$ and an angle $\theta$.

Rodrigues’ formula maps $(\hat{\omega},\theta)\rightarrow R$:
$$R(\hat{\omega},\theta)=I+\sin\theta\,[\hat{\omega}]_\times+(1-\cos\theta)\,[\hat{\omega}]_\times^2$$


In [ ]:
def skew(w):
    wx, wy, wz = w
    return np.array([[0, -wz, wy],
                     [wz, 0, -wx],
                     [-wy, wx, 0]])

def rodrigues(w_hat, theta):
    w_hat = np.asarray(w_hat, dtype=float)
    w_hat = w_hat / np.linalg.norm(w_hat)
    W = skew(w_hat)
    I = np.eye(3)
    return I + np.sin(theta)*W + (1 - np.cos(theta))*(W @ W)

def axis_angle_from_R(R, eps=1e-8):
    R = np.asarray(R, dtype=float)
    cos_theta = (np.trace(R) - 1.0) / 2.0
    cos_theta = np.clip(cos_theta, -1.0, 1.0)
    theta = np.arccos(cos_theta)

    if theta < 1e-6:
        return np.array([1.0, 0.0, 0.0]), 0.0

    if np.pi - theta < 1e-6:
        diag = np.diag(R)
        i = np.argmax(diag)
        w = np.zeros(3)
        w[i] = np.sqrt(max((diag[i] + 1.0) / 2.0, 0.0))

        if i == 0:
            w[1] = (R[0,1] + R[1,0]) / (4*w[0] + eps)
            w[2] = (R[0,2] + R[2,0]) / (4*w[0] + eps)
        elif i == 1:
            w[0] = (R[0,1] + R[1,0]) / (4*w[1] + eps)
            w[2] = (R[1,2] + R[2,1]) / (4*w[1] + eps)
        else:
            w[0] = (R[0,2] + R[2,0]) / (4*w[2] + eps)
            w[1] = (R[1,2] + R[2,1]) / (4*w[2] + eps)

        n = np.linalg.norm(w)
        w = w / n if n > eps else np.array([1.0, 0.0, 0.0])
        return w, theta

    w = np.array([
        R[2,1] - R[1,2],
        R[0,2] - R[2,0],
        R[1,0] - R[0,1]
    ]) / (2.0*np.sin(theta))

    w = w / np.linalg.norm(w)
    return w, theta

w_hat, theta = axis_angle_from_R(R)
R_from_axis = rodrigues(w_hat, theta)

print("Recovered axis w_hat =", np.round(w_hat, 6))
print("Recovered theta (deg) =", np.round(np.rad2deg(theta), 6))
print("||R - R_from_axis|| =", np.linalg.norm(R - R_from_axis))

## 3) Quaternions
We use unit quaternion $q=[w,x,y,z]$.
We’ll do $R \rightarrow q$ and $q \rightarrow R$.


In [ ]:
def quat_from_axis_angle(w_hat, theta):
    w_hat = np.asarray(w_hat, dtype=float)
    w_hat = w_hat / np.linalg.norm(w_hat)
    half = theta / 2.0
    qw = np.cos(half)
    qv = w_hat * np.sin(half)
    return np.array([qw, qv[0], qv[1], qv[2]])

def R_from_quat(q):
    q = np.asarray(q, dtype=float)
    q = q / np.linalg.norm(q)
    w, x, y, z = q
    return np.array([
        [1 - 2*(y*y + z*z),     2*(x*y - z*w),     2*(x*z + y*w)],
        [    2*(x*y + z*w), 1 - 2*(x*x + z*z),     2*(y*z - x*w)],
        [    2*(x*z - y*w),     2*(y*z + x*w), 1 - 2*(x*x + y*y)]
    ])

def quat_from_R(R):
    R = np.asarray(R, dtype=float)
    tr = np.trace(R)
    if tr > 0:
        S = np.sqrt(tr + 1.0) * 2
        qw = 0.25 * S
        qx = (R[2,1] - R[1,2]) / S
        qy = (R[0,2] - R[2,0]) / S
        qz = (R[1,0] - R[0,1]) / S
    else:
        if (R[0,0] > R[1,1]) and (R[0,0] > R[2,2]):
            S = np.sqrt(1.0 + R[0,0] - R[1,1] - R[2,2]) * 2
            qw = (R[2,1] - R[1,2]) / S
            qx = 0.25 * S
            qy = (R[0,1] + R[1,0]) / S
            qz = (R[0,2] + R[2,0]) / S
        elif R[1,1] > R[2,2]:
            S = np.sqrt(1.0 + R[1,1] - R[0,0] - R[2,2]) * 2
            qw = (R[0,2] - R[2,0]) / S
            qx = (R[0,1] + R[1,0]) / S
            qy = 0.25 * S
            qz = (R[1,2] + R[2,1]) / S
        else:
            S = np.sqrt(1.0 + R[2,2] - R[0,0] - R[1,1]) * 2
            qw = (R[1,0] - R[0,1]) / S
            qx = (R[0,2] + R[2,0]) / S
            qy = (R[1,2] + R[2,1]) / S
            qz = 0.25 * S
    q = np.array([qw, qx, qy, qz])
    return q / np.linalg.norm(q)

q = quat_from_R(R)
R_from_q = R_from_quat(q)

print("Quaternion q=[w,x,y,z] =", np.round(q, 6))
print("||R - R_from_q|| =", np.linalg.norm(R - R_from_q))

q2 = quat_from_axis_angle(w_hat, theta)
R_from_q2 = R_from_quat(q2)
print("q (from axis-angle) =", np.round(q2, 6))
print("||R - R_from_q2|| =", np.linalg.norm(R - R_from_q2))

## 4) Visualization: axis–angle meaning
We draw:
- space frame axes (gray)
- recovered axis $\hat{\omega}$ (orange)
- a vector $v$ (black)
- rotated vector $Rv$ (blue)


In [ ]:
v = np.array([0.3, 0.8, 0.1])
v_rot = R @ v

fig = plt.figure(figsize=(7,6))
ax = fig.add_subplot(111, projection="3d")

draw_frame(ax, np.eye(3), name="s", axis_len=1.0, colors=("gray","gray","gray"), lw=2, alpha=0.8)
draw_arrow(ax, 1.0*w_hat, color="tab:orange", label=r"$\hat{\omega}$", lw=4)

draw_arrow(ax, v, color="black", label="v", lw=3)
draw_arrow(ax, v_rot, color="tab:blue", label="R v", lw=3)

ax.set_title(r"Same rotation shown via axis–angle (recovered from R)")
ax.set_xlabel("X"); ax.set_ylabel("Y"); ax.set_zlabel("Z")
set_axes_equal_3d(ax, lim=1.4)
plt.show()

print("theta (deg) =", np.round(np.rad2deg(theta), 3))

## 5) Optional: save a GIF rotating about the recovered axis
This animates $v(t)=R(\hat{\omega},t)v$ from $t=0$ to $t=\theta$.

Set `OUT_DIR` to where you want the gif saved.


In [ ]:
OUT_DIR = Path('.')  # default: current directory
# Example for repo layout (notebook in notebooks/): OUT_DIR = Path('..') / 'assets'

OUT_DIR.mkdir(parents=True, exist_ok=True)
out_path = OUT_DIR / 'r_axis_angle_demo.gif'

n = 60
ts = np.linspace(0, theta, n)

fig = plt.figure(figsize=(6,5))
ax = fig.add_subplot(111, projection='3d')

def update(i):
    ax.cla()
    draw_frame(ax, np.eye(3), name='s', axis_len=1.0, colors=('gray','gray','gray'), lw=2, alpha=0.8)
    draw_arrow(ax, 1.0*w_hat, color='tab:orange', label='axis ω', lw=4)

    Rt = rodrigues(w_hat, ts[i])
    vt = Rt @ v
    draw_arrow(ax, v,  color='black', lw=3, label='v')
    draw_arrow(ax, vt, color='tab:blue', lw=3, label='R(t) v')

    ax.set_title(f'Rotate about ω: {np.rad2deg(ts[i]):.1f}° / θ={np.rad2deg(theta):.1f}°')
    ax.set_xlabel('X'); ax.set_ylabel('Y'); ax.set_zlabel('Z')
    set_axes_equal_3d(ax, lim=1.4)

ani = FuncAnimation(fig, update, frames=n, interval=60)
ani.save(str(out_path), writer='pillow', fps=20)
plt.close(fig)

print('Saved:', out_path)